# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [42]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [43]:
path = (r'C:\Users\A454\Desktop\python-api-challenge\WeatherPy\clean_weather_data.csv')
clean_weather_data = pd.read_csv(path)
clean_weather_data.head()

,City,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date
0,Mananjary,-21.2167,48.3333,71.13,77,56,4.12,MG,1657701742
1,Bilma,18.6853,12.9164,96.57,14,97,11.01,NE,1657702015
2,Caravelas,-17.7125,-39.2481,67.68,90,5,2.86,BR,1657702015
3,Martapura,-3.4167,114.8500,82.49,74,40,5.75,ID,1657702015
4,Tuktoyaktuk,69.4541,-133.0374,35.87,96,100,11.43,CA,1657702015


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [44]:
gmaps.configure(api_key=g_key)

#locations_df = clean_weather_data[["Latitude", "Longitude"]]
#locations_df.head()

locations = []
for index, rows in clean_weather_data.iterrows():
    locations.append((rows.Latitude, rows.Longitude))
locations    


In [45]:
locations = clean_weather_data[["Latitude", "Longitude"]]
weights = clean_weather_data["Humidity (%)"]

In [46]:
fig = gmaps.figure()
fig

Figure(layout=FigureLayout(height='420px'))

In [47]:
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

Narrow down the DataFrame to find your ideal weather condition. For example:


A max temperature lower than 55 degrees but higher than 50.


Wind speed less than 10 mph.


<25% cloudiness.


Drop any rows that don't satisfy all three conditions. You want to be sure the weather is ideal.


Note: Feel free to adjust your specifications, but make sure to limit the number of rows returned by your API requests to a 
reasonable number.

In [48]:
hotel_df = clean_weather_data.loc[clean_weather_data["Humidity (%)"] <= 15]
hotel_df


,City,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date
1,Bilma,18.6853,12.9164,96.57,14,97,11.01,NE,1657702015
23,Rawah,34.4816,41.9093,103.42,15,0,17.54,IQ,1657702017
53,Tabas,33.5959,56.9244,97.02,11,0,9.22,IR,1657702021
141,Adrar,20.5022,-10.0711,95.54,14,11,8.41,MR,1657702031
142,Cayenne,4.9333,-52.3333,73.44,1,100,2.30,GF,1657702031
148,Hami,42.8000,93.4500,96.30,12,76,9.48,CN,1657702032
342,Riyadh,24.6877,46.7219,104.14,7,100,15.88,SA,1657701793
421,Kununurra,-15.7667,128.7333,76.98,13,11,8.05,AU,1657702065
425,Korla,41.7597,86.1469,90.46,14,89,7.49,CN,1657702065
470,Esfahan,32.6572,51.6776,92.84,6,0,18.79,IR,1657701956


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [49]:
hotel_df["Hotel Name"] = ""
hotel_df

C:\Users\A454\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,City,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date,Hotel Name
1,Bilma,18.6853,12.9164,96.57,14,97,11.01,NE,1657702015,
23,Rawah,34.4816,41.9093,103.42,15,0,17.54,IQ,1657702017,
53,Tabas,33.5959,56.9244,97.02,11,0,9.22,IR,1657702021,
141,Adrar,20.5022,-10.0711,95.54,14,11,8.41,MR,1657702031,
142,Cayenne,4.9333,-52.3333,73.44,1,100,2.30,GF,1657702031,
148,Hami,42.8000,93.4500,96.30,12,76,9.48,CN,1657702032,
342,Riyadh,24.6877,46.7219,104.14,7,100,15.88,SA,1657701793,
421,Kununurra,-15.7667,128.7333,76.98,13,11,8.05,AU,1657702065,
425,Korla,41.7597,86.1469,90.46,14,89,7.49,CN,1657702065,
470,Esfahan,32.6572,51.6776,92.84,6,0,18.79,IR,1657701956,


In [50]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row

# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [41]:
# Add marker layer ontop of heat map
marker = gmaps.marker_layer(locations, info_box_content=hotel_info)


fig.add_layer(marker)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))